In [1]:
import datetime, requests, re, time, traceback, pandas as pd, numpy as np
from bs4 import BeautifulSoup

# header для замаскированного запроса
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36"
}

In [2]:
df = pd.DataFrame()
url_project = "https://www.fl.ru/projects/5218684/trebuetsya-razrabotat-logotip-biol-technology-za-konkurs-30000--14500-za-soglasovannyie-dorabotki.html"

# Целевой конкурс
response = requests.get(url_project, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

df.loc[url_project, "Название конкурса"] = soup.find_all("span")[7].get_text().strip() # Название
df.loc[url_project, "Заказчик конкурса"] = soup.find_all("a")[70].get_text().strip() # Заказчик

for user_block in soup.find_all('a'):
    if '/users' in str(user_block):
        break
if df.loc[url_project, "Заказчик конкурса"] == 'Заказчик':
    df.loc[url_project, "Заказчик конкурса"] = user_block.get('href').split('/')[-1]

df.loc[url_project, "Дата регистрации конкурса"] = soup.find("div", class_="text-gray-opacity-4 d-inline-block text-5 cursor-default").get_text().strip()[1:11]
budget_raw = soup.find_all("span")[9].get_text().strip()
budget = ""
for i in range(len(budget_raw)):
    if "0" <= budget_raw[i] <= "9":
        budget += budget_raw[i]
budget = int(budget)
df.loc[url_project, "Бюджет"] = budget # Бюджет
df.loc[url_project, "Описание"] = soup.find("div", class_="text-5 text-dark").get_text().strip() # Описание
df.loc[url_project, "URL заказчика"] = "https://www.fl.ru" + soup.find_all("a")[70].get("href") # URL заказчика
chapter = ""
for elem in soup.find_all("a", class_="text-decoration-none link-hover-danger text-blue cursor-pointer"):
    chapter += elem.get_text() + "/"
if chapter == "":
    for elem in soup.find_all("a", class_="1 text-decoration-none link-hover-danger text-blue cursor-pointer"):
        chapter += elem.get_text() + "/"
chapter = chapter[:-1]
df.loc[url_project, "Разделы"] = chapter # Разделы
df.loc[url_project, "Участники"] = int(soup.find("span", attrs={"id": "stat-freelancers"}).get_text()) # Кол-во участников
df.loc[url_project, "Кандидаты"] = int(soup.find("span", attrs={"id": "stat-candidates"}).get_text()) # Кандидаты
df.loc[url_project, "В бане"] = int(soup.find("span", attrs={"id": "stat-banned"}).get_text()) # Кол-во заблокированных пользователей
df.loc[url_project, "Дата завершения приема заявок"] = soup.find("div", attrs={"class": "contest-period-in"}).find_all("p")[0].get_text()[-10:] # Дата завершения приема заявок
df.loc[url_project, "Заказ закрыт"] = "Заказ закрыт" in response.text # Закрыт ли заказ

if 'coments-list' in response.text:
    df.loc[url_project, "Число ветвлений комментариев"] = int(soup.find("span", attrs={"id": "co-all"}).get_text()) # Число ветвлений комментариев
    df.loc[url_project, "Тексты комментариев"] = soup.find_all('ul', attrs={'class':'coments-list'})[0].get_text() # Тексты комментариев
    df.loc[url_project, "Число комментариев"] = len(re.findall(r'\[\d+\.\d+\.\d+ \| \d+:\d+\]', 
                                                              soup.find_all('ul', attrs={'class':'coments-list'})[0].get_text())) # Число комментариев
else:
    df.loc[url_project, "Число ветвлений комментариев"] = 0
    df.loc[url_project, "Число комментариев"] = 0


# Заказчик целевого конкурса

response2 = requests.get(df.loc[url_project, "URL заказчика"], headers=headers) # Делаем замаскированный запрос на страницу заказчика

df.loc[url_project, "Платный аккаунт"] = "Платный аккаунт" in response2.text # Платный ли аккаунт
df.loc[url_project, "Телефон подтвержден"] = "Телефон подтвержден" in response2.text # подтвержден ли телефон
df.loc[url_project, "Email подтвержден"] = "Email подтвержден" in response2.text # подтвержден ли email
df.loc[url_project, "Паспортные данные проверены"] = "Паспортные данные проверены" in response2.text # проверены ли паспортные данные
df.loc[url_project, "Данные юридического лица/ИП проверены"] = "Данные юридического лица/ИП проверены" in response2.text # проверены ли данные юридического лица/ИП

for block in BeautifulSoup(response2.text).find_all('td', class_="b-layout__td"):
    if "На сайте " in block.get_text():
        break
block = block.get_text()

if re.findall(r'[\d]+ [лг]+', block) != []:
    df.loc[url_project, "Лет на сайте"] = int(re.findall(r'[\d]+ [лг]+', block)[0].strip().split(' ')[0])
else:
    df.loc[url_project, "Лет на сайте"] = 0

if re.findall(r'[\d]+ [м]+', block) != []:
    df.loc[url_project, "Месяцев на сайте"] = int(re.findall(r'[\d]+ [м]+', block)[0].strip().split(' ')[0])
else:
    df.loc[url_project, "Месяцев на сайте"] = 0

if re.findall(r'На сайте .+\t', block)[0].strip() == "На сайте меньше месяца":
    df.loc[url_project, "Лет на сайте"] = 0
    df.loc[url_project, "Месяцев на сайте"] = 0

for block in BeautifulSoup(response2.text).find_all('td', class_="b-layout__td"):
    if "Рейтинг" in block.get_text() and "Отзывы" in block.get_text():
        break
block = re.sub(r'\s', r' ', block.get_text()).strip()
block = re.sub(r'[  ]+', r' ', block)
df.loc[url_project, "Рейтинг"] = float(re.findall(r'Рейтинг *\d*\.*\d+', block)[0].split(' ')[-1]) if 'Рейтинг' in block else 0
df.loc[url_project, "Безопасные сделки"] = int(re.findall(r'Безопасные сделки \d+', block)[0].split(' ')[-1]) if 'Безопасные сделки' in block else 0
df.loc[url_project, "Выбран[а] исполнителем"] = int(re.findall(r'Выбран[а] исполнителем \d+', block)[0].split(' ')[-1]) if 'Выбран' in block else 0
df.loc[url_project, "Отзывы+"] = int(re.findall(r'\+ \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0
df.loc[url_project, "Отзывы-"] = int(re.findall(r'- \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0
df.loc[url_project, "Дата обращения"] = datetime.date.today().strftime("%d.%m.%y")

In [3]:
today = datetime.date.today().strftime("%Y%m%d")
df.to_excel(f'FL_Конкурсы_{today}.xlsx', sheet_name="FL_Конкурсы") # Сохраняем наш датафрейм в Excel файл

In [4]:
# Собираем промо-характеристики конкурсов
df = pd.DataFrame()
i = 1
while True:
    url_smmr = f'https://www.fl.ru/konkurs/page-{i}/'
    response = requests.get(url_smmr, headers=headers)
    if BeautifulSoup(response.text).find_all('div', attrs={'id':'projects-list'}) != []: # Если на странице есть конкурсы
        blocks = str(BeautifulSoup(response.text).find_all('div', attrs={'id':'projects-list'})[0]\
        .find_all('div', attrs={'class':'b-post'})).split('b-post ') # список блоков, содержащих промо-характеристики конкурсов и их URL адреса
        
        # Проход по каждому из блоков
        
        for block in blocks:
            try:
                url_project = 'https://www.fl.ru' + re.findall(r'/projects/\d+/', block)[0]
                df.loc[url_project, 'Закреп конкурса'] = 'post_pin' in block
                df.loc[url_project, 'Срочность конкурса'] = 'urgently-1.png' in block
                df.loc[url_project, 'Цвет конкурса'] = '_bg_ffffd4' in block
            except Exception as exc:
                pass
    else:
        break
    i += 1

In [5]:
# Собираем характеристики всех конкурсов и заказчиков
for url_project in df.index:
    print(url_project)
    time.sleep(0.25) # делаем задержку по времени, чтобы обращение к серверу не было подозрительным 
    response = requests.get(url_project, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    df.loc[url_project, "Название конкурса"] = soup.find_all("span")[7].get_text().strip() # Название
    df.loc[url_project, "Заказчик конкурса"] = soup.find_all("a")[70].get_text().strip() # Заказчик
    
    for user_block in soup.find_all('a'):
        if '/users' in str(user_block):
            break
    if df.loc[url_project, "Заказчик конкурса"] == 'Заказчик':
        df.loc[url_project, "Заказчик конкурса"] = user_block.get('href').split('/')[-1]

    df.loc[url_project, "Дата регистрации конкурса"] = soup.find("div", class_="text-gray-opacity-4 d-inline-block text-5 cursor-default").get_text().strip()[1:11]
    budget_raw = soup.find_all("span")[9].get_text().strip()
    budget = ""
    for i in range(len(budget_raw)):
        if "0" <= budget_raw[i] <= "9":
            budget += budget_raw[i]
    budget = int(budget)
    df.loc[url_project, "Бюджет"] = budget # Бюджет
    df.loc[url_project, "Описание"] = soup.find("div", class_="text-5 text-dark").get_text().strip() # Описание
    df.loc[url_project, "URL заказчика"] = "https://www.fl.ru" + soup.find_all("a")[70].get("href") # URL заказчика
    chapter = ""
    for elem in soup.find_all("a", class_="text-decoration-none link-hover-danger text-blue cursor-pointer"):
        chapter += elem.get_text() + "/"
    if chapter == "":
        for elem in soup.find_all("a", class_="1 text-decoration-none link-hover-danger text-blue cursor-pointer"):
            chapter += elem.get_text() + "/"
    chapter = chapter[:-1]
    df.loc[url_project, "Разделы"] = chapter # Разделы
    df.loc[url_project, "Участники"] = int(soup.find("span", attrs={"id": "stat-freelancers"}).get_text()) # Кол-во участников
    df.loc[url_project, "Кандидаты"] = int(soup.find("span", attrs={"id": "stat-candidates"}).get_text()) # Кандидаты
    df.loc[url_project, "В бане"] = int(soup.find("span", attrs={"id": "stat-banned"}).get_text()) # Кол-во заблокированных пользователей
    df.loc[url_project, "Дата завершения приема заявок"] = soup.find("div", attrs={"class": "contest-period-in"}).find_all("p")[0].get_text()[-10:] # Дата завершения приема заявок
    df.loc[url_project, "Заказ закрыт"] = "Заказ закрыт" in response.text # Закрыт ли заказ
    
    if 'coments-list' in response.text:
        df.loc[url_project, "Число ветвлений комментариев"] = int(soup.find("span", attrs={"id": "co-all"}).get_text()) # Число ветвлений комментариев
        df.loc[url_project, "Тексты комментариев"] = soup.find_all('ul', attrs={'class':'coments-list'})[0].get_text() # Тексты комментариев
        df.loc[url_project, "Число комментариев"] = len(re.findall(r'\[\d+\.\d+\.\d+ \| \d+:\d+\]', 
                                                                  soup.find_all('ul', attrs={'class':'coments-list'})[0].get_text())) # Число комментариев
    else:
        df.loc[url_project, "Число ветвлений комментариев"] = 0
        df.loc[url_project, "Число комментариев"] = 0
        


        
    time.sleep(0.25)
    response2 = requests.get(df.loc[url_project, "URL заказчика"], headers=headers) # Делаем замаскированный запрос на страницу заказчика

    df.loc[url_project, "Платный аккаунт"] = "Платный аккаунт" in response2.text # Платный ли аккаунт
    df.loc[url_project, "Телефон подтвержден"] = "Телефон подтвержден" in response2.text # подтвержден ли телефон
    df.loc[url_project, "Email подтвержден"] = "Email подтвержден" in response2.text # подтвержден ли email
    df.loc[url_project, "Паспортные данные проверены"] = "Паспортные данные проверены" in response2.text # проверены ли паспортные данные
    df.loc[url_project, "Данные юридического лица/ИП проверены"] = "Данные юридического лица/ИП проверены" in response2.text # проверены ли данные юридического лица/ИП

    for block in BeautifulSoup(response2.text).find_all('td', class_="b-layout__td"):
        if "На сайте " in block.get_text():
            break
    block = block.get_text()
        
    if re.findall(r'[\d]+ [лг]+', block) != []:
        df.loc[url_project, "Лет на сайте"] = int(re.findall(r'[\d]+ [лг]+', block)[0].strip().split(' ')[0])
    else:
        df.loc[url_project, "Лет на сайте"] = 0

    if re.findall(r'[\d]+ [м]+', block) != []:
        df.loc[url_project, "Месяцев на сайте"] = int(re.findall(r'[\d]+ [м]+', block)[0].strip().split(' ')[0])
    else:
        df.loc[url_project, "Месяцев на сайте"] = 0

    if re.findall(r'На сайте .+\t', block)[0].strip() == "На сайте меньше месяца":
        df.loc[url_project, "Лет на сайте"] = 0
        df.loc[url_project, "Месяцев на сайте"] = 0

    for block in BeautifulSoup(response2.text).find_all('td', class_="b-layout__td"):
        if "Рейтинг" in block.get_text() and "Отзывы" in block.get_text():
            break
    block = re.sub(r'\s', r' ', block.get_text()).strip()
    block = re.sub(r'[  ]+', r' ', block)
    df.loc[url_project, "Рейтинг"] = float(re.findall(r'Рейтинг *\d*\.*\d+', block)[0].split(' ')[-1]) if 'Рейтинг' in block else 0
    df.loc[url_project, "Безопасные сделки"] = int(re.findall(r'Безопасные сделки \d+', block)[0].split(' ')[-1]) if 'Безопасные сделки' in block else 0
    df.loc[url_project, "Выбран[а] исполнителем"] = int(re.findall(r'Выбран[а] исполнителем \d+', block)[0].split(' ')[-1]) if 'Выбран' in block else 0
    df.loc[url_project, "Отзывы+"] = int(re.findall(r'\+ \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0
    df.loc[url_project, "Отзывы-"] = int(re.findall(r'- \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0
    df.loc[url_project, "Дата обращения"] = datetime.date.today().strftime("%d.%m.%y")
df

https://www.fl.ru/projects/5234732/
https://www.fl.ru/projects/5239362/
https://www.fl.ru/projects/5238868/
https://www.fl.ru/projects/5238564/
https://www.fl.ru/projects/5238166/
https://www.fl.ru/projects/5238042/
https://www.fl.ru/projects/5238035/
https://www.fl.ru/projects/5237052/
https://www.fl.ru/projects/5236716/
https://www.fl.ru/projects/5236014/
https://www.fl.ru/projects/5236413/
https://www.fl.ru/projects/5235461/
https://www.fl.ru/projects/5236198/
https://www.fl.ru/projects/5236134/
https://www.fl.ru/projects/5236047/
https://www.fl.ru/projects/5236019/
https://www.fl.ru/projects/5236015/
https://www.fl.ru/projects/5235788/
https://www.fl.ru/projects/5235768/
https://www.fl.ru/projects/5235591/
https://www.fl.ru/projects/5235469/
https://www.fl.ru/projects/5235452/
https://www.fl.ru/projects/5235342/
https://www.fl.ru/projects/5235286/
https://www.fl.ru/projects/5234993/
https://www.fl.ru/projects/5234985/
https://www.fl.ru/projects/5234984/
https://www.fl.ru/projects/5

,Закреп конкурса,Срочность конкурса,Цвет конкурса,Название конкурса,Заказчик конкурса,Дата регистрации конкурса,Бюджет,Описание,URL заказчика,Разделы,...,Паспортные данные проверены,Данные юридического лица/ИП проверены,Лет на сайте,Месяцев на сайте,Рейтинг,Безопасные сделки,Выбран[а] исполнителем,Отзывы+,Отзывы-,Дата обращения
https://www.fl.ru/projects/5234732/,False,True,True,"Разработка фирменного стиля, пиццерии и бургер...",Александр Головлев,11.10.2023,30000.0,ВНИМАНИЕ! к слову Ранчо добавлены символы К3. ...,https://www.fl.ru/users/Mplata,Дизайн и Арт,...,False,False,10.0,7.0,4172.56,22.0,0.0,29.0,0.0,20.10.23
https://www.fl.ru/projects/5239362/,False,False,False,разработка логотипа (торгового знака) и цветов...,rvserkov,20.10.2023,5000.0,необходимо разработать логотип (торговый знак)...,https://www.fl.ru/users/rvserkov,Дизайн и Арт/Логотипы,...,False,False,0.0,0.0,184.33,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5238868/,False,False,False,Нейминг для студии лазерной эпиляции,darina-terehova,19.10.2023,5000.0,Добрый день. Нужна ваша помощь для создания н...,https://www.fl.ru/users/darina-terehova,Тексты/Слоганы/Нейминг,...,False,False,3.0,1.0,392.00,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5238564/,False,False,False,Дизайн настенного квартального календаря.,gamma-nn99,19.10.2023,5000.0,Создать дизайн-макет настенного квартального к...,https://www.fl.ru/users/gamma-nn99,Дизайн и Арт/Полиграфический дизайн,...,False,True,3.0,2.0,321.55,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5238166/,False,False,False,Дизайн значков и автобусов городского обществе...,Ульяна Дудникова,18.10.2023,10000.0,Разработка дизайна металлических значков на од...,https://www.fl.ru/users/pu-200632,Дизайн и Арт,...,False,False,2.0,11.0,193.33,0.0,0.0,0.0,0.0,20.10.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.fl.ru/projects/5213321/,False,True,False,Отрисовка комикса в национальной тематике Даге...,Ольга Руновская,24.08.2023,5000.0,Первый выпуск комикса на 12 страниц. Формат А ...,https://www.fl.ru/users/runovskaya43,Анимация и флеш/Раскадровки,...,False,False,1.0,8.0,296.23,1.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5213057/,False,False,False,Дизайн Главной страницы корпоративного портала...,ekaterina_shuva,23.08.2023,20000.0,Добрый день! Мы объявляем конкурс на дизайн Г...,https://www.fl.ru/users/ekaterina_shuva,Разработка сайтов/Дизайн сайтов,...,False,True,0.0,1.0,1724.33,1.0,0.0,1.0,0.0,20.10.23
https://www.fl.ru/projects/5212399/,False,False,False,Нейминг. Придумайте название для компании/сайт...,artwide,22.08.2023,10000.0,Мы создаём собственный сайт дополнительно к ма...,https://www.fl.ru/users/artwide,Тексты/Слоганы/Нейминг,...,False,False,8.0,5.0,923.96,6.0,0.0,14.0,0.0,20.10.23
https://www.fl.ru/projects/5212152/,False,False,False,Разработка логотипа для строительной компании,Максим Верховых,22.08.2023,8000.0,Необходимо разработать логотип и фирменный сти...,https://www.fl.ru/users/sales7571,Дизайн и Арт/Логотипы,...,False,True,0.0,1.0,1097.33,3.0,0.0,3.0,0.0,20.10.23


In [6]:
today = datetime.date.today().strftime("%Y%m%d")
df.to_excel(f'FL_Конкурсы_{today}.xlsx', sheet_name="FL_Конкурсы")

In [7]:
slice_df = df[df["Бюджет"] > df["Бюджет"].mean()] # все конкурсы с бюджетом выше среднего
slice_df

,Закреп конкурса,Срочность конкурса,Цвет конкурса,Название конкурса,Заказчик конкурса,Дата регистрации конкурса,Бюджет,Описание,URL заказчика,Разделы,...,Паспортные данные проверены,Данные юридического лица/ИП проверены,Лет на сайте,Месяцев на сайте,Рейтинг,Безопасные сделки,Выбран[а] исполнителем,Отзывы+,Отзывы-,Дата обращения
https://www.fl.ru/projects/5234732/,False,True,True,"Разработка фирменного стиля, пиццерии и бургер...",Александр Головлев,11.10.2023,30000.0,ВНИМАНИЕ! к слову Ранчо добавлены символы К3. ...,https://www.fl.ru/users/Mplata,Дизайн и Арт,...,False,False,10.0,7.0,4172.56,22.0,0.0,29.0,0.0,20.10.23
https://www.fl.ru/projects/5236014/,False,False,False,Разработка дизайна на пакет: Сдобу Пасхальную ...,ilyabodyagin,13.10.2023,50000.0,На этапе конкурса необходимо: Представить диза...,https://www.fl.ru/users/ilyabodyagin,Полиграфия/Дизайн упаковки,...,True,False,6.0,3.0,2277.01,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5236413/,False,False,False,Создание и ведение сайта Fan Кейтеринг,sokolovdmitry77,14.10.2023,50000.0,Здравствуйте! Требуется создать сайт для нашей...,https://www.fl.ru/users/sokolovdmitry77,Разработка сайтов/Дизайн сайтов,...,False,False,0.0,4.0,188.33,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5236134/,False,False,False,Народный листинг!,Vladimir Kuznetsov,13.10.2023,777777.0,Откровенно говоря – наболело... Вот посмотрите...,https://www.fl.ru/users/vovchansky,Тексты,...,False,False,15.0,4.0,3782.84,0.0,0.0,21.0,0.0,20.10.23
https://www.fl.ru/projects/5236015/,False,False,False,Разработка дизайна на пакета: Кулич Творожный,ilyabodyagin,13.10.2023,50000.0,На этапе конкурса необходимо: Представить диза...,https://www.fl.ru/users/ilyabodyagin,Полиграфия/Дизайн упаковки,...,True,False,6.0,3.0,2277.01,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5235469/,False,True,True,брендбук фирменный стиль ресторана Головлёвъ,Александр Головлев,12.10.2023,32000.0,"ресторан Головлёвъ это семейный ресторан, с жи...",https://www.fl.ru/users/Mplata,Дизайн и Арт/Фирменный стиль/Дизайн и Арт/Лого...,...,False,False,10.0,7.0,4172.56,22.0,0.0,29.0,0.0,20.10.23
https://www.fl.ru/projects/5234993/,False,False,False,Разработать дизайн интерьера ресторана Головлевъ,Александр Головлев,11.10.2023,150000.0,Требуется реализовать дизайн интерера Головлев...,https://www.fl.ru/users/Mplata,Дизайн и Арт/Интерьеры,...,False,False,10.0,7.0,4172.56,22.0,0.0,29.0,0.0,20.10.23
https://www.fl.ru/projects/5233285/,False,False,True,Изготовить дизайн одежды для спорт клуба,rotshelda,07.10.2023,35000.0,Техническое задание для дизайнеров на создание...,https://www.fl.ru/users/rotshelda,Дизайн и Арт,...,False,False,3.0,9.0,4715.08,2.0,0.0,2.0,0.0,20.10.23
https://www.fl.ru/projects/5232304/,False,False,False,Разработка дизайна логотипа и этикетки мороженого,mariakluc7,05.10.2023,40000.0,Нужно разработать дизайн логотипа и этикетки д...,https://www.fl.ru/users/mariakluc7,Дизайн и Арт/Дизайн упаковки,...,False,False,0.0,0.0,192.33,0.0,0.0,0.0,0.0,20.10.23
https://www.fl.ru/projects/5232273/,False,False,False,ДИЗАЙН ЭТИКЕТКИ НОВОГО ВИНА,eletsko,05.10.2023,50000.0,Алкогольная компания «Альтаир» объявляет конку...,https://www.fl.ru/users/eletsko,Дизайн и Арт/Фирменный стиль,...,False,False,3.0,4.0,2359.68,0.0,0.0,0.0,0.0,20.10.23
